In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
from xgboost import XGBRegressor
from xgboost import plot_importance

from sklearn.preprocessing import LabelEncoder
from itertools import product


import time

In [2]:
test = pd.read_csv('../data/test.csv', index_col=0)
submit = pd.read_csv('../data/sample_submission.csv')
data = pd.read_pickle('data.pkl')


In [3]:
data.columns

Index(['month_idx', 'shop_id', 'item_id', 'item_cnt_month', 'cat_id',
       'type_code', 'item_cnt_month_lag_1', 'item_cnt_month_lag_2',
       'item_cnt_month_lag_3', 'item_cnt_month_lag_6', 'item_cnt_month_lag_12',
       'month', 'days', 'item_shop_first_sale', 'item_first_sale'],
      dtype='object')

In [4]:
data = data[[
        'month_idx', 'shop_id', 'item_id', 'item_cnt_month', 'cat_id',
           'type_code', 'item_cnt_month_lag_1', 'item_cnt_month_lag_2',
       'item_cnt_month_lag_3', 'item_cnt_month_lag_6', 'item_cnt_month_lag_12',
       'month', 'days', 'item_shop_first_sale', 'item_first_sale'
]]


In [5]:
data.month_idx.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 23], dtype=int8)

In [6]:
X_train = data[data.month_idx < 21].drop(['item_cnt_month'], axis=1)
y_train = data[data.month_idx < 21]['item_cnt_month']
X_valid = data[data.month_idx == 21].drop(['item_cnt_month'], axis=1)
y_valid  = data[data.month_idx == 21]['item_cnt_month']
X_test = data[data.month_idx ==23].drop(['item_cnt_month'], axis=1)


In [7]:
del data

In [8]:
import gc


In [9]:
gc.collect()

66

In [10]:

ts = time.time()

model = XGBRegressor(
    max_depth=8,
    n_estimators = 1000,
    min_child_weight = 300,
    colsample_bytree=0.8,
    eta=0.3,
    seed=42
)

model.fit(X_train, y_train,eval_metric="rmse", eval_set=[(X_train, y_train),(X_valid, y_valid)],
    verbose=True,
    early_stopping_rounds=10)

time.time() - ts


[0]	validation_0-rmse:2.49200	validation_1-rmse:2.89566
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:2.30098	validation_1-rmse:2.74394
[2]	validation_0-rmse:2.19197	validation_1-rmse:2.64726
[3]	validation_0-rmse:2.13475	validation_1-rmse:2.58928
[4]	validation_0-rmse:2.09526	validation_1-rmse:2.54150
[5]	validation_0-rmse:2.07371	validation_1-rmse:2.51534
[6]	validation_0-rmse:2.05337	validation_1-rmse:2.51094
[7]	validation_0-rmse:2.04216	validation_1-rmse:2.51032
[8]	validation_0-rmse:2.03277	validation_1-rmse:2.51322
[9]	validation_0-rmse:2.02795	validation_1-rmse:2.50790
[10]	validation_0-rmse:2.01633	validation_1-rmse:2.50524
[11]	validation_0-rmse:2.01351	validation_1-rmse:2.50179
[12]	validation_0-rmse:2.01039	validation_1-rmse:2.49839
[13]	validation_0-rmse:2.00578	validation_1-rmse:2.49552
[14]	validation_0-rmse:2.00237	validation_1-rmse:2.50

15.521493911743164

In [11]:
y_pred = model.predict(X_valid)
y_test = model.predict(X_test)

submission = pd.DataFrame({
    "ID": test.index,
    "item_cnt_month": y_test
})

submission.to_csv('xgb_submission.csv', index=False )


In [17]:
submit

,0,100
0,1,100
1,2,100
2,3,100
3,4,100
4,5,100
...,...,...
3054,3055,100
3055,3056,100
3056,3057,100
3057,3058,100


In [16]:
submission

,ID,item_cnt_month
0,0,0.360681
1,1,0.000000
2,2,0.383577
3,3,1.242619
4,4,2.760417
...,...,...
3055,3055,0.765410
3056,3056,0.000894
3057,3057,0.000894
3058,3058,0.000894
